In [1]:
from pyspark import *

In [2]:
sc = SparkContext(master="local", appName="test")

In [3]:
sc

<SparkContext master=local appName=test>

In [13]:
#download les data 
# --> http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
import urllib
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'data_10_percent.gz')
data_file = './data_10_percent.gz'

lines = sc.textFile(data_file)

lines


./data_10_percent.gz MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [14]:
#initialiser un SQLContext
sqlContext = SQLContext(sc)

sqlContext

In [20]:
#parser les data
datas = sqlContext.read.json(lines)
datas.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|0,tcp,http,SF,181...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,235...|
|0,tcp,http,SF,219...|
|0,tcp,http,SF,217...|
|0,tcp,http,SF,217...|
|0,tcp,http,SF,212...|
|0,tcp,http,SF,159...|
|0,tcp,http,SF,210...|
|0,tcp,http,SF,212...|
|0,tcp,http,SF,210...|
|0,tcp,http,SF,177...|
|0,tcp,http,SF,222...|
|0,tcp,http,SF,256...|
|0,tcp,http,SF,241...|
|0,tcp,http,SF,260...|
|0,tcp,http,SF,241...|
|0,tcp,http,SF,257...|
|0,tcp,http,SF,233...|
|0,tcp,http,SF,233...|
+--------------------+
only showing top 20 rows



In [28]:
# a l'aide de la methode StructType structurer un schema de donnees  
from pyspark.sql.types import *

parts = lines.map(lambda l: l.split(","))
parts.show()
people = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "test"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
schema

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [8]:
#instencier un objet DataFrame à l'aide de votre schema 


DataFrame[duration: int, protocol_type: string, service: string, flag: string, src_bytes: int, dst_bytes: int, interactions: string]

In [9]:
#afficher les 10premieres lignes du DataFrame


+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s

In [10]:
#creer une vue 'interactions' 
# a quoi servent les TempView?


In [11]:
#selectionner les colonnes duration, dst_bytes ou le protocole est tcp et les duration > 1000 avec des dst_bytes nuls


In [12]:
#compter le nombre de lignes qui satisfont la condition ci-dessus


139

In [13]:
#afficher les lignes ci-dessus en utilisant la fonction lambda  


Duration: 5057, Dest. bytes: 0
Duration: 5059, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5041, Dest. bytes: 0
Duration: 5056, Dest. bytes: 0
Duration: 5064, Dest. bytes: 0
Duration: 5043, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5049, Dest. bytes: 0
Duration: 5061, Dest. bytes: 0
Duration: 5048, Dest. bytes: 0
Duration: 5047, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5068, Dest. bytes: 0
Duration: 5062, Dest. bytes: 0
Duration: 5046, Dest. bytes: 0
Duration: 5052, Dest. bytes: 0
Duration: 5044, Dest. bytes: 0
Duration: 5054, Dest. bytes: 0
Duration: 5039, Dest. bytes: 0
Duration: 5058, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5032, Dest. bytes: 0
Duration: 5063, Dest. bytes: 0
Duration: 5040, Dest. bytes: 0
Duration: 5051, Dest. bytes: 0
Duration: 5066, Dest. bytes: 0
Duration

**Dans la suite de l'exercice vous utiliserez la fonction time de la librairie time afin de mesurer le temps d'execution des requêtes**

In [14]:
#compter le nombre d'interaction par protocol 


+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Times to execute the request : 12.495s


In [15]:
#selectionner les colonnes duration, dst_bytes ou le protocole est tcp et les duration > 1000 avec des dst_bytes nuls
# utiliser la methode filter


+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Requete executee en 12.522 secondes


In [16]:
#afficher votre resultats


['Duration: 0, Dest. bytes: 5450',
 'Duration: 0, Dest. bytes: 486',
 'Duration: 0, Dest. bytes: 1337',
 'Duration: 0, Dest. bytes: 1337',
 'Duration: 0, Dest. bytes: 2032']

utiliser la fonction `udf` afin de 'wrapper' la fonction ci-dessous et de 'labeliser' le dataset. 
```python 
from pyspark.sql.functions import udf


def attack_or_normal_func(s):
    return "normal" if s == "normal." else "attack"

```

In [17]:
#à l'aide de la fonction udf ajouter une colonne label au DataFrame 


In [18]:
#afficher le schema de donnees et verifier que la colonne label est bien ajoutee


root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- interactions: string (nullable = true)
 |-- label: string (nullable = true)



In [19]:
#compter les labels


+------+------+
| label| count|
+------+------+
|normal| 97278|
|attack|396743|
+------+------+

Requete executee en 12.223 secondes


In [20]:
#compter les labels par protocoles 


+------+-------------+------+
| label|protocol_type| count|
+------+-------------+------+
|normal|          udp| 19177|
|normal|         icmp|  1288|
|normal|          tcp| 76813|
|attack|         icmp|282314|
|attack|          tcp|113252|
|attack|          udp|  1177|
+------+-------------+------+

Requete executee en 10.293 secondes
